In [1]:
import firebase_admin
from firebase_admin import firestore
import os
import pandas as pd
from pathlib import Path
import sys
sys.path.append(os.path.dirname(os.getcwd() ))

import visualize


In [2]:
cred = firebase_admin.credentials.Certificate('key.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()
instructions_ref = db.collection('instructions')
verify_ref = db.collection('verification')


In [3]:
list_instruction = list(instructions_ref.get())
list_verification = list(verify_ref.get())
len(list_instruction), len(list_verification)


(6924, 9803)

In [4]:
instructions_ref_sandbox = db.collection('instructions_sandbox')
verify_ref_sandbox = db.collection('verification_sandbox')

list_instruction_sandbox = list(instructions_ref_sandbox.get())
list_verification_sandbox = list(verify_ref_sandbox.get())
len(list_instruction_sandbox), len(list_verification_sandbox)

(54, 364)

In [4]:
# Qualified workers
current_dir = os.getcwd()
parent_dir = Path(current_dir).parent.absolute()
path_qualified = os.path.join(parent_dir, "qualified_workers.csv")

qualified_work_list = pd.read_csv(path_qualified, header=None)[0].tolist()

qualified_work_list

['1',
 '1A',
 'AY7WPVKHVNBLG',
 'A1198W1SPF1R4',
 'A3QAL3X23LHJRV',
 'A3GWRDHAURRNK6',
 'A2DDPSXH2X96RF',
 'ALYR5CI2SM2JC',
 'A320QA9HJFUOZO',
 'APRZ7BR8C0ZMQ',
 'A3862RIFFUV141',
 'A1AKL5YH9NLD2V',
 'A2WX434EAQOE29',
 'AYIFHDQSXQJ6B',
 'A21LONLNBOB8Q',
 'A3UENPLNM9AQBK',
 'A5WWHKD82I8UE']

In [ ]:
# # Takes a long time!!!!  ~10m

# # Set qualified instructions as RVS_excellent  - UPDATES DATA!!!
# qualified_instructions = [
#   e.to_dict()['key'] for e in list_instruction if (
#     e.to_dict()['work_id'] in qualified_work_list) ]

# for id in qualified_instructions:
#   instructions_ref.document(id).update(
#     {'review': 'RVS_excellent'})


In [ ]:
# # Split instructions to train, dev, test sets.
# import json

# instruction_dir = "/mnt/hackney/human_data"

# def save_split(split_name, split_set):

#   split_path = os.path.join(instruction_dir, split_name)
#   if os.path.exists(split_path):
#     os.remove(split_path)
#   with open(split_path, 'a') as outfile:
#     for sample in split_set:
#       json.dump(sample, outfile, default=lambda o: o.__dict__)
#       outfile.write('\n')
#       outfile.flush()

#   print (f"Size of {split_name} set:{len(split_set)} saved to: {split_path}")


# # Save set splits

# dev_ids = ['X']
# test_ids = ['X']


# test_set =  [
#   x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in test_ids]

# save_split("test.json", test_set)

# dev_set =  [
#   x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in dev_ids]

# save_split("dev.json", dev_set)

# train_set = [
#   x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] not in dev_ids+test_ids]

# save_split("train.json", train_set)


In [5]:
# check the new workers ID
df_instruction_manhattan = pd.DataFrame(
    [
        x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and  'date_start' in x.to_dict() and '2022-11-01' <x.to_dict()['date_start'] ])

df_verification = pd.DataFrame(
        [x.to_dict() for x in list_verification] )


df_manhattan = df_instruction_manhattan.merge(df_verification, left_on='key', right_on='key_instruction')

work_ids = list(set(df_manhattan['work_id_x'].to_list()))


for work_id in work_ids:
    if work_id in qualified_work_list:
        continue
    sample_worker = df_manhattan[df_manhattan['work_id_x']==work_id]
    if sample_worker.shape[0]<3:
        continue
    print (f"--------{work_id}--------")
    print (sample_worker[['date_start_x', 'review', 'assignmentId_x','dist_m']])


--------A3NFGEUZAH9V5G--------
                    date_start_x review                  assignmentId_x  \
3097  2023-02-21 13:56:33.808331    NaN  38BQUHLA9W1L1I24V6KLN6RNZS9MO8   
3098  2023-03-02 20:47:27.417651    NaN  3JRJSWSMQHMY35H5VACAMVO3WSZ3E5   
3099  2023-03-02 20:47:27.417651    NaN  3JRJSWSMQHMY35H5VACAMVO3WSZ3E5   
3100  2023-02-21 20:50:18.604267    NaN  3KKG4CDWKIZ2Q59SND174MFYI7U94O   
3101  2023-02-21 20:50:18.604267    NaN  3KKG4CDWKIZ2Q59SND174MFYI7U94O   
3102  2023-02-21 20:50:18.604267    NaN  3KKG4CDWKIZ2Q59SND174MFYI7U94O   
3103  2023-02-20 01:21:26.925471    NaN  3PS7W85Z8Z3M7QOI2E2FCURQXTET9G   
3104  2023-03-02 20:42:37.710061    NaN  3OCHAWUVGOLD53GTBLU1R6QCNKUKX4   
3105  2023-03-02 20:42:37.710061    NaN  3OCHAWUVGOLD53GTBLU1R6QCNKUKX4   
3106  2023-03-02 20:45:21.637037    NaN  39L1G8WVWQSZJ4NTJMHVFB597JV31M   

      dist_m  
3097     0.0  
3098   220.0  
3099    91.0  
3100   293.0  
3101   191.0  
3102   327.0  
3103    85.0  
3104     5.0  
3105   2

In [6]:
# Check the stack of RVS_excellent
all_data = pd.DataFrame([x.to_dict() for x in list_instruction ])
x = all_data[all_data['review']=='RVS_excellent' ]
print (f"RVS_excellent: {x[x['verified_n']==0].shape[0]}")
# invalid
invalid = all_data[all_data['valid']==False]
print (f"invalid: {invalid[invalid['verified_n']<=5].shape[0]}")




RVS_excellent: 30
invalid: 79


In [7]:
# merge instructions with verification
df_instruction = pd.DataFrame([x.to_dict() for x in list_instruction])
df_ver = pd.DataFrame([x.to_dict() for x in list_verification ])

instruction_ver = df_instruction.merge(df_ver, left_on='key', right_on='key_instruction')


In [8]:
# Valid instruction
dist_threshold = 100


manhattan_instruction_ver = instruction_ver[instruction_ver['region']=='Manhattan']
pittsburgh_instruction_ver = instruction_ver[instruction_ver['region']=='Pittsburgh']


all_valid_instructions = instruction_ver[instruction_ver['dist_m']<dist_threshold]
manhattan_valid_instructions = all_valid_instructions[all_valid_instructions['region']=='Manhattan']
pittsburgh_valid_instructions = all_valid_instructions[all_valid_instructions['region']=='Pittsburgh']


valid_instructions = all_valid_instructions.groupby('key_instruction').min()
# all_instructions = instruction_ver.groupby('key_x').min()
len_instructions = instruction_ver.drop_duplicates(subset=['key_x']).shape[0]
print (f"All valid for theshold {dist_threshold}: {round(100*valid_instructions.shape[0]/len_instructions, 3)}" )


manhattan_valid_instructions = manhattan_valid_instructions.groupby('key_instruction').min()
len_manhattan_instructions = manhattan_instruction_ver.drop_duplicates(subset=['key_x']).shape[0]
print (f"Manhattan valid for theshold {dist_threshold}: {round(100*manhattan_valid_instructions.shape[0]/len_manhattan_instructions, 3)}" )

pittsburgh_valid_instructions = pittsburgh_valid_instructions.groupby('key_instruction').min()
len_pittsburgh_instructions = pittsburgh_instruction_ver.drop_duplicates(subset=['key_x']).shape[0]
print (f"Pittsburgh valid for theshold {dist_threshold}: {round(100*pittsburgh_valid_instructions.shape[0]/len_pittsburgh_instructions, 3)}" )

# All valid for theshold 100: 96.906
# Manhattan valid for theshold 100: 97.22
# Pittsburgh valid for theshold 100: 95.103




All valid for theshold 100: 97.106
Manhattan valid for theshold 100: 97.439
Pittsburgh valid for theshold 100: 95.201


In [9]:
# status of colection

print (f"size of Manhattan-set: {len_manhattan_instructions}")
print (f"size of Pittsburgh-set: {len_pittsburgh_instructions}")

size of Manhattan-set: 5856
size of Pittsburgh-set: 1021


In [10]:
# assure all points are of the same format  - UPDATES DATA!!!
import math
import util

list_instruction_all_x = [x.to_dict() for x in list_instruction]
for sample in list_instruction_all_x: 
  for key, point in sample.items():
    FLAG_prob = False
    if not 'point' in key:
      continue
      

    if isinstance(point, list):
      if not(isinstance(point[0], float) and isinstance(point[1], float)):
        FLAG_prob = True
      y = float(point[0])
      x = float(point[1])
      point = [y ,x]
    else:
      FLAG_prob = True
      point = util.point_from_str_point(point)
      y = point.y
      x = point.x
      point = [y, x]
      if not (y>40 and x<-73):
        if x>40 and y<-73:
          point = [x, y]

    assert isinstance(y, float) and isinstance(x, float), print (type(y), type(x))
    assert (isinstance(point, list) and point[0]>40 and point[1]<-73), print(point )

    id = sample['key']
    # instructions_ref.document(id).update(
    #       {key: point})
    if FLAG_prob:
      print (f"Writng task PROB with {key}: {sample['key']} ")


# verify 
list_verification_x = [x.to_dict() for x in list_verification]
for sample in list_verification_x: 
  for key, point in sample.items():
    FLAG_prob = False
    if not 'point' in key:
      continue
    
    if isinstance(point, list):
      if isinstance(point[0], float) and isinstance(point[1], float):
        if (isinstance(point, list) and point[0]>40 and point[1]<-73):
          continue
        else:
          FLAG_prob = True
      else:
        FLAG_prob = True
      y = float(point[0])
      x = float(point[1])
      point = [y ,x]
    else:
      FLAG_prob = True
      point = util.point_from_str_point(point)
      y = point.y
      x = point.x
      point = [y, x]
      if not (y>40 and x<-73):
        if x>40 and y<-73:
          point = [x, y]

    assert isinstance(y, float) and isinstance(x, float), print (type(y), type(x))
    assert (isinstance(point, list) and point[0]>40 and point[1]<-73), print(f"point: {point} key: {sample['key']}")
    
    if 'key' not in sample:
      print (sample['assignmentId'])
      continue
    
    if FLAG_prob:
      print (f"Verify PROB with {key}: {sample['key']} ")
      id = sample['key']
      # verify_ref.document(id).update(
      #       {key: point})

    

In [11]:
import util
import math

# UPDATE!!!
content = "marked in red"

# Check all distances are correct 
for i in range(0,instruction_ver.shape[0]):
  sample = instruction_ver.iloc[i]
  if sample['rvs_goal_point_y']!=sample['rvs_goal_point_x']:
    point_goal_y = util.point_from_list_coord_yx(sample['rvs_goal_point_y'])
    point_goal_x = util.point_from_list_coord_yx(sample['rvs_goal_point_x'])
    dist = round(util.get_distance_between_points(point_goal_y, point_goal_x))
    if dist>1:
      print (f"goal point - key: {sample['key_x']}| {sample['rvs_goal_point_x']}!= {sample['rvs_goal_point_y']}")
      id = sample['key_y']
      verify_ref.document(id).update(
          {'rvs_goal_point': sample['rvs_goal_point_x']})
    
    if not isinstance(sample['rvs_goal_point_y'], list) and math.isnan(sample['rvs_goal_point_y']): 
      id = sample['key_y']
      # verify_ref.document(id).update(
      #     {'rvs_goal_point': sample['rvs_goal_point_x']})

 
  if sample['rvs_start_point_x']!=sample['rvs_start_point_y']:
    point_start_y = util.point_from_list_coord_yx(sample['rvs_start_point_y'])
    point_start_x = util.point_from_list_coord_yx(sample['rvs_start_point_x'])

    dist = round(util.get_distance_between_points(point_start_y, point_start_x))

    if dist==0:
      continue
    print (f"start point - key: {sample['key_x']}| {sample['rvs_start_point_x']}!= {sample['rvs_start_point_y']}")

    print (f"dist: {dist}")
    id = sample['key_y']
    verify_ref.document(id).update(
    {'rvs_start_point': sample['rvs_start_point_x']})


list_instruction_all_x = [x.to_dict() for x in list_instruction]
dict_col = {'assignmentId': str, 'content': str, 'date_finish': str, 'date_start': str, 'hit_id': str, 
'work_id': str, 'rvs_sample_number': str, 'rvs_path': str, 'rvs_goal_point': list, 'rvs_start_point': list, 
'task': int, 'key': str, 'valid': bool, 'verified_n': int, 'region': str, 'review': str}
should_be_coulmns = set(dict_col.keys())

for sample in list_instruction_all_x: 
  keys = set(sample.keys())
  difference = keys.difference(should_be_coulmns)
  if len(difference)>0:
    if len(difference)==1 and 'review' in difference:
      continue
    print (difference)
  if content in sample['content']:
    print (sample)

  for key, value in sample.items():
    if not isinstance(value, dict_col[key]):
      print (f"key: {key}. value: {value} {type(value)}")


  
# calc distance

list_verification_x = [x.to_dict() for x in list_verification]
print (list_verification_x[0].keys())
for sample in list_verification_x: 
  prediction = util.point_from_list_coord_yx(sample['predict_goal_point'])
  if 'rvs_goal_point' not in sample:
    if 'key_instruction' in sample:
      rvs_goal_point = df_instruction[df_instruction['key_x']==sample['key_instruction']]['rvs_goal_point_x']
      print ("rvs_goal_point: ", rvs_goal_point)
    else:
      found_matches = df_instruction[
        (
          df_instruction['rvs_path']==str(sample['rvs_path'])) & (
            df_instruction['rvs_sample_number']==str(sample['rvs_sample_number']))]
      assert found_matches.shape[0] <= 1, print(found_matches.shape[0])
      if found_matches.shape[0]==0:
        print (f"No such instruction: {sample['assignmentId']}| {sample['rvs_sample_number']}")
      else:
        print (f"one connection: {found_matches['key'].iloc[0]}")
        id = sample['key']
        # verify_ref.document(id).update(
        #   {'key_instruction': found_matches['key'].iloc[0]})


      
    # print (f"NO rvs_goal_point. \n assignmentId: {sample['assignmentId']}")
    continue 
  point_goal = util.point_from_list_coord_yx(sample['rvs_goal_point'])

  dist_true = round(util.get_distance_between_points(prediction, point_goal))
  if 'dist_m' not in sample:
    print (f"No dist_m. \n assignmentId: {sample['assignmentId']}")
    id = sample['key']
    # verify_ref.document(id).update(
    #       {'dist_m': dist_true})
    continue
  dist_m = sample['dist_m']

  if dist_true != dist_m:
    print (f"dist_true: {dist_true} | dist_m: {dist_m}")
    id = sample['key']
    verify_ref.document(id).update(
          {'dist_m': dist_true})



dict_keys(['rvs_sample_number', 'key', 'rvs_goal_point', 'predict_goal_point', 'key_instruction', 'work_id', 'task', 'assignmentId', 'hit_id', 'date_start', 'dist_m', 'rvs_path', 'date_finish', 'rvs_start_point'])


In [12]:
# Takes a long time ~10m
# Check verified_n

for index, sample in instruction_ver.iterrows():
  written_verified_n = sample['verified_n']
  key = sample['key_x']
  true_verified_n = instruction_ver[instruction_ver['key_instruction']==key].shape[0]
  if written_verified_n!=true_verified_n:
    print (f"written_verified_n:{written_verified_n}, true_verified_n:{true_verified_n}")
    instructions_ref.document(key).update(
          {'verified_n': true_verified_n})

written_verified_n:4, true_verified_n:1
written_verified_n:1, true_verified_n:2
written_verified_n:1, true_verified_n:2


In [ ]:

# # Check incorrect instructions

# def get_end_point_tag(row):
#   path_geodata = row['rvs_path_x']
#   sample_n = int(row['rvs_sample_number_x'])
#   final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
#   _, _, _, entity, _ = visualize.get_maps_and_instructions(path=final_path_geodata, specific_sample=sample_n)[0]

#   main_tag = entity.geo_landmarks['end_point'].main_tag

  
#   return main_tag in row['content']

# invalid = instruction_ver[instruction_ver['valid_x']==False]
# invalid_verified = invalid[invalid['verified_n']==0]   

# invalid_verified['is_landmarks']  = invalid_verified.apply(get_end_point_tag, axis=1)

# invalid_verified_end_point_not_mentioned = invalid_verified[invalid_verified['is_landmarks']==False]

# print (invalid_verified_end_point_not_mentioned.shape)





In [ ]:
# for i in set(invalid_verified_end_point_not_mentioned[invalid_verified_end_point_not_mentioned['dist_m']<200]['key_x'].tolist()):
#     print (i)

In [ ]:
# # Check incorrect instructions 2


# invalid = instruction_ver[instruction_ver['valid_x']==False]
# invalid_under = invalid[invalid['dist_m']<=2000]   

# set_invalid_under = set(invalid_under[invalid_under['dist_m']>1000]['key_x'].tolist())
# print (len(set_invalid_under))

# for i in set_invalid_under:
#     print (i)


In [46]:
# # Takes a long time!!!!
# for sample in list_instruction_all_x:
    
#   path_data = sample['rvs_path'].replace("/app_instructor/", "")

#   current_dir = os.getcwd()
#   parent_dir = Path(current_dir).parent.absolute()
#   path_instructions = os.path.join(parent_dir, path_data)
   
#   entities = util.load_entities(path_instructions)
#   assert len(entities)>0, print (sample)


In [23]:
# # Takes a long time!!!!

# # Check unique points in Manhattan
# # 2854 4583

# import json
# import pathlib 
# from collections import Counter


# def add_osmid(row):
#   path_geodata = row['rvs_path_x']
#   sample_n = int(row['rvs_sample_number_x'])
#   final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
#   _, _, _, entity, _ = visualize.get_maps_and_instructions(path=final_path_geodata, specific_sample=sample_n)[0]
#   goal_osmid = entity.geo_landmarks['end_point'].osmid
#   start_osmid = entity.geo_landmarks['start_point'].osmid

#   return str(goal_osmid) + str(start_osmid)
    

# manhattan_instruction_ver['goal__start_osmid']  = manhattan_instruction_ver.apply(add_osmid, axis=1, result_type='expand')


# points = {}
# def first(row):
#   rvs_goal_point_x = row['rvs_goal_point_x']
#   rvs_start_point_x = row['rvs_start_point_x']

#   point = str(rvs_goal_point_x) + str(rvs_start_point_x)
#   if point not in points:
#         points[point]=1
#   else:
#       points[point]+=1

# manhattan_instruction_ver.apply(first, axis=1) #.groupby(['rvs_goal_point_x', 'rvs_start_point_x']).first()
# print (len(points), manhattan_instruction_ver.shape[0])

# counter = {}
# for k, v in points.items():
#     if v not in counter:
#         counter[v] = 1
#     else:
#         counter[v] += 1

# counter_sorted = sorted(counter.items(), key=lambda x:x[0])
# print (counter_sorted)




# unique_osmids = manhattan_instruction_ver['goal__start_osmid'].tolist()
# counter_osmids = Counter(unique_osmids)

# print (f"found {len(counter_osmids)} unique paths")
# path = pathlib.Path().resolve().parent.parent / "cabby/geo/unique_osmids.json"
# with open(path, 'w') as fp:
#     json.dump(counter_osmids, fp)



In [13]:

all_valid_instructions_with_flase_validation = all_valid_instructions[all_valid_instructions['valid_x']==False]
#Update valid instructions
valid_instructions_tmp = [
  e.to_dict()['key'] for e in list_instruction if (
   e.to_dict()['key'] in all_valid_instructions_with_flase_validation['key_x'].tolist()) ]

print(len(valid_instructions_tmp))
for id in valid_instructions_tmp:
  instructions_ref.document(id).update(
    {'valid': True})

0


In [14]:
#How much was not verified at all
print("verified_n==0 ", df_instruction[df_instruction['verified_n']==0].shape[0])
print("valid==Flase ", df_instruction[df_instruction['valid']==False].shape[0])

# verified_n==0  103
# valid==Flase  1054

verified_n==0  44
valid==Flase  80


In [14]:
# # Check number of completed assignments per HIT

# hit_id = "A2WX434EAQOE293EQVJH0T40MC0C87AOL51STM3H3TH93"
# assignments = [
#   e.to_dict()['key'] for e in list_instruction if (
#    e.to_dict()['hit_id']==hit_id) ]

# print(f"number of assignments for hit id {hit_id}: {len(assignments)}")


number of assignments for hit id A2WX434EAQOE293EQVJH0T40MC0C87AOL51STM3H3TH93: 0


In [35]:
# visualize geosample with instruction by key

instruction = df_instruction[df_instruction['key']=='A21LONLNBOB8Q3W0KKJIARRAK4IHRDBD7NODRB3S8K32'].iloc[0]
path_geodata = instruction['rvs_path']
sample_n = int(instruction['rvs_sample_number'])
final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
map_osm, _, _, _, _ = visualize.get_maps_and_instructions(path=final_path_geodata, specific_sample=sample_n, with_path=False)[0]
print (instruction['content'])
map_osm


I'm not that far northeast of you on Smallman Street. Meet me here at the garden across from the street corner with the museum. Eide's Entertainment toy shop is directly south of me, and Level 7 bar is to the southwest. 


In [29]:
# visualize geosample with instruction by worker id (last instruction)

instruction = df_instruction[df_instruction['work_id']=='A320QA9HJFUOZO'].sort_values(
    by=['date_finish'], ascending=False).iloc[9]
print (instruction)
path_geodata = instruction['rvs_path']
sample_n = int(instruction['rvs_sample_number'])
final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
map_osm, _, _, _, _ = visualize.get_maps_and_instructions(path=final_path_geodata, specific_sample=sample_n, with_path=False)[0]
print (instruction['content'])
map_osm

rvs_sample_number                                                  151
content              Go north and meet me at the irish pub on West ...
rvs_path               /app_instructor/data/manhattan_samples_v86.gpkg
review                                                             NaN
key                      A320QA9HJFUOZO3LB1BGHFL2XYM7YWKOPTNSOPTN0YTH3
rvs_goal_point                               [40.7522208, -73.9918556]
verified_n                                                           0
work_id                                                 A320QA9HJFUOZO
region                                                       Manhattan
valid                                                            False
hit_id                                  3LB1BGHFL2XYM7YWKOPTNSOPTN0YTH
rvs_start_point                              [40.7422116, -73.9893974]
task                                                               3.0
assignmentId                            3VAR3R6G1P26GT0015A2IOQSMYR8O6
date_s

In [34]:
# Map error

voc = ['map', 'error', 'invalid']
condition = df_instruction['content'].str.contains('|'.join(voc)) 

instruction = df_instruction[condition]
instruction[['key', 'content']]



,key,content
4607,A5WWHKD82I8UE31YWE12TE0DMO4BF2P0QZIPY1317XV3,Meet me at the parking space off of 20th Stree...
4690,A5WWHKD82I8UE3HA5ODM5KATG3DQCQDXJ6OO9881VS23,Meet meet at the waste basket just off 5th Ave...
4774,A5WWHKD82I8UE3ULIZ0H1VA6ZBZC3730M6P715LN51O3,Meet me at the recycling container just off of...
4796,A5WWHKD82I8UE3XU9MCX6VODV57SXNED3E9M2P7F2R13,Meet me at the bar inside New York Penn Statio...


In [46]:

# Takes a long time!!!!  ~4m

# TO DO - load file and append only data not in there. 

# Save instructions to data/human


current_dir = os.getcwd()
parent_dir = Path(current_dir).parent.absolute()
path_instructions = os.path.join(parent_dir, "data/human/instructions.json")


visualizae_map  = {}

set_paths = all_data['rvs_path'].drop_duplicates()

for path_geodata in set_paths:
  final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
  visualizae_map[path_geodata] = util.load_entities(final_path_geodata)


def add_landmarks(row):
  path_geodata = row['rvs_path']
  sample_n = int(row['rvs_sample_number'])
  entity = visualizae_map[path_geodata][sample_n]

  landmarks = {l_t:[util.list_yx_from_point(l.geometry), l.main_tag]  for l_t, l in entity.geo_landmarks.items() if l.geometry}
  return landmarks

print ("start going over data")  

data_to_save = all_data
data_to_save['landmarks']  = data_to_save.apply(add_landmarks, axis=1)

# Save instructions to data/human

instuction_col_del = ["work_id", "hit_id", "assignmentId", "date_start", "date_finish", "review"]
data_to_save.drop(instuction_col_del, axis=1).to_json(path_instructions, lines=True, orient='records')
print (f"{data_to_save.shape[0]} instructions saved => {path_instructions}")

# Save verifications to data/human
current_dir = os.getcwd()
parent_dir = Path(current_dir).parent.absolute()
path_verification = os.path.join(parent_dir, "data/human/verification.json")

verification_col_del = ["work_id", "hit_id", "assignmentId", "date_start", "date_finish", "valid"]
df_verification.drop(verification_col_del, axis=1).to_json(path_verification, lines=True, orient='records')
print (f"{df_verification.shape[0]} verifications saved => {path_verification}")


start going over data
6924 instructions saved => /home/onlp_gcp_biu/tzuf/cabby/app/data/human/instructions.json
9803 verifications saved => /home/onlp_gcp_biu/tzuf/cabby/app/data/human/verification.json
